## `rocky`

In [3]:
import sys
# mydir = r'C:\Users\aweaver\rocky\rocky-app\src\py'
mydir = './src'
sys.path.append(mydir)
from rocky.core import rocky
from rocky.triangle import Triangle

import pandas as pd
import numpy as np

In [4]:
# creat a new rocky object
r = rocky()

# inspect object
print(r)

rocky(id='rocky0', mod=(), f=(), plot=(), t=())


In [5]:
## Could read in sample data (taylor_ashe, mack_1994) to try it out
r.SampleTri('taylor_ashe')
# r.paid_loss.reset_base_linear_model(var_name='accident_period', trends=False)

## Could copy a triangle from a file to the clipboard and then read it in
# r.FromClipboard('paid_loss')

## for this demo, we'll just read in a triangle from a link ratio file

# triangle file path for WC 4Q2022
# path = r'O:\STAFFHQ\SYMDATA\Actuarial\Reserving Applications\IBNR Allocation\4Q2022 Analysis\Workers Compensation 4Q2022.xlsb'

# read in triangle from file
# r.FromExcel(path, 1, 'paid_loss', 'Gross Paid Ultimates', "a28:u48") # takes filepath, number of origin columns, name of triangle, sheet name, and range of triangle
print(r)

AttributeError: 'Triangle' object has no attribute 'base_linear_model'

In [4]:
# access triangle object
r.paid_loss.tri

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
r.paid_loss.acc.dt.year

0    2001
1    2002
2    2003
3    2004
4    2005
5    2006
6    2007
7    2008
8    2009
9    2010
Name: accident_period, dtype: int32

In [6]:
r.AddModel('glm', 'glm', 'paid_loss')
r

rocky(id='rocky0', mod="glm", f=(), plot=(), t="paid_loss")

In [7]:
# access linear design matrix:
r.paid_loss.X_base.head(10)

,is_observed,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_0002,development_period_0003,development_period_0004,development_period_0005,development_period_0006,development_period_0007,development_period_0008,development_period_0009,development_period_0010
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0


In [8]:
# quick ATA factor summary
r.paid_loss.ata_summary()

Age-to-Age Factors as of (months),1,2,3,4,5,6,7,8,9
2001,3.143,1.543,1.278,1.238,1.209,1.044,1.04,1.063,1.018
2002,3.511,1.755,1.545,1.133,1.084,1.128,1.057,1.086,
2003,4.448,1.717,1.458,1.232,1.037,1.12,1.061,,
2004,4.568,1.547,1.712,1.073,1.087,1.047,,,
2005,2.564,1.873,1.362,1.174,1.138,,,,
2006,3.366,1.636,1.369,1.236,,,,,
2007,2.923,1.878,1.439,,,,,,
2008,3.953,2.016,,,,,,,
2009,3.619,,,,,,,,
Vol Wtd,,,,,,,,,


In [9]:
# Triangle data type actually has a robust chain-ladder implementation that can be accessed

# this isn't important for right now, but it is there and it works well:
r.paid_loss.ult('vwa', 5, tail=1.061)

Accident Period
2001-01-01    4139452.0
2002-01-01    5765176.0
2003-01-01    5706935.0
2004-01-01    5621078.0
2005-01-01    5154550.0
2006-01-01    5422953.0
2007-01-01    5961413.0
2008-01-01    7197996.0
2009-01-01    6120629.0
2010-01-01    5011529.0
Name: Chain Ladder Ultimate Loss, dtype: float64

In [10]:
# now add a GLM model to the triangle:
r.AddModel('icrfs', 'loglinear', 'paid_loss')
print(r)

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.
rocky(id='rocky0', mod=("glm", "icrfs"), f=(), plot=(), t="paid_loss")


In [11]:
r.icrfs.GetWeights().head()

,weights
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [12]:
r.icrfs.GetY('train')

0     12.787864
1     12.771722
2     12.579383
3     12.646287
4     13.001686
5     12.889503
6     12.996419
7     12.792414
8     12.839167
9     12.748438
10    13.550164
11    13.692236
12    13.817308
13    13.918293
14    13.449059
15    13.750529
16    13.650201
17    13.875333
18    13.802028
20    13.322102
21    13.747118
22    13.738866
23    13.562151
24    13.807461
25    13.650044
26    13.938965
27    14.182491
30    13.087648
31    13.983808
32    13.832027
33    14.261734
34    13.553481
35    13.598644
36    13.876859
40    13.175574
41    13.007502
42    13.528916
43    12.515328
44    13.132019
45    13.467314
50    13.261078
51    12.679184
52    11.897664
53    12.771537
54    13.061847
60    11.893702
61    13.176480
62    13.114315
63    12.237019
70    11.849040
71    12.491898
72    12.543990
80    12.333714
81    12.959953
90    11.126498
Name: paid_loss, dtype: float64

In [13]:
# r.icrfs.TuneFitHyperparameters()
r.icrfs.SetHyperparameters(alpha=3, l1_ratio=0.05)
r.icrfs

loglinear(alpha=3.0, l1_ratio=0.05)

In [14]:
# r.glm.TuneFitHyperparameters()
r.glm.SetHyperparameters(alpha=0.2, power=2.3)
r.glm

tweedieGLM(alpha=0.2, power=2.3)

In [15]:
r.icrfs.GetY('train')
# r.icrfs.is_fitted

0     12.787864
1     12.771722
2     12.579383
3     12.646287
4     13.001686
5     12.889503
6     12.996419
7     12.792414
8     12.839167
9     12.748438
10    13.550164
11    13.692236
12    13.817308
13    13.918293
14    13.449059
15    13.750529
16    13.650201
17    13.875333
18    13.802028
20    13.322102
21    13.747118
22    13.738866
23    13.562151
24    13.807461
25    13.650044
26    13.938965
27    14.182491
30    13.087648
31    13.983808
32    13.832027
33    14.261734
34    13.553481
35    13.598644
36    13.876859
40    13.175574
41    13.007502
42    13.528916
43    12.515328
44    13.132019
45    13.467314
50    13.261078
51    12.679184
52    11.897664
53    12.771537
54    13.061847
60    11.893702
61    13.176480
62    13.114315
63    12.237019
70    11.849040
71    12.491898
72    12.543990
80    12.333714
81    12.959953
90    11.126498
Name: paid_loss, dtype: float64

In [16]:
r.icrfs

loglinear(alpha=3.0, l1_ratio=0.05)

In [17]:
## now that we have a set of hyperparameters, we can reuse them
## next time using model.SetHyperparameters(alpha, power):
# -> r.tweedie.SetHyperparameters(alpha=0.4, power=1.8)

# fit base model to the data
r.icrfs.Fit()
r.glm.Fit()
print(r)

rocky(id='rocky0', mod=("glm", "icrfs"), f=(), plot=(), t="paid_loss")


In [18]:
dat = r.icrfs.GetX("train")
DYwGp = r.icrfs.GetHeteroGp().loc[r.icrfs.GetIdx('train')]
DYwGp.rename(columns={DYwGp.columns[0]: "gp"}, inplace=True)
DYw = r.icrfs.GetWeights("train")
#this is a test


# print(f"dat: {dat.head(10)}")
# print(f"DYwGp: {DYwGp.head(10)}")

# join weight groups to data frame
# dat = dat.join(DYwGp)
# dat

In [19]:
r.icrfs.GetWeights("train")

,weights
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0


In [20]:
resid2 = r.icrfs.RawResiduals().apply(lambda x: x**2)
resid2 

0     3.310067e+01
1     3.364652e+02
2     6.792692e+03
3     4.269626e+04
4     1.555093e+05
5     3.828523e+05
6     6.866264e+05
7     9.481833e+05
8     1.062302e+06
9     1.062489e+06
10    3.085135e+02
11    1.535919e+04
12    1.651347e+05
13    9.138839e+05
14    3.198039e+06
15    7.728825e+06
16    1.375646e+07
17    1.892005e+07
18    2.117676e+07
20    1.577666e+05
21    1.536668e+06
22    8.290674e+06
23    2.873925e+07
24    6.922090e+07
25    1.230007e+08
26    1.690583e+08
27    3.601782e+04
30    4.025690e+07
31    1.391704e+08
32    3.349092e+08
33    5.948043e+08
34    1.142243e+05
35    2.375418e+06
36    2.240357e+07
40    2.449794e+05
41    4.993060e+06
42    4.690121e+07
43    2.504820e+08
44    8.650097e+08
45    3.959297e+05
50    1.031792e+07
51    9.672562e+07
52    5.618094e+05
53    1.125712e+07
54    5.600656e+05
dtype: float64

In [21]:
r.icrfs.GetDegreesOfFreedom()

36

In [22]:
W_vec = r.icrfs.GetWeights(kind="train")

# get the diagonal matrix of weights
W = np.diag(W_vec.values.flatten())

# get the X matrix (design matrix)
X = r.icrfs.GetX(kind="train").values

# get the process variance
# if process_var_est == "ube":
V = r.icrfs._ProcessVarUBE()
# elif process_var_est == "mle":
#     V = r.icrfs._ProcessVarMLE()
# else:
#     raise ValueError("process_var_est must be 'ube' or 'mle'")

# # calculate the variance matrix
# varMatrix = (np.linalg.inv(W) - X @ np.linalg.inv(X.T @ W @ X) @ X.T) * V

# # return the standard errors
# se = np.sqrt(np.abs(np.diag(varMatrix)))
# return pd.Series(se, index=r.icrfs.GetX(kind="train").columns, name="Std Error")

In [23]:
r.icrfs.RawResiduals().dropna()

0         5.753318
1       -18.342986
2       -82.417787
3      -206.630735
4      -394.346644
5      -618.750574
6      -828.629211
7      -973.747030
8     -1030.680221
9     -1030.770951
10      -17.564553
11     -123.932203
12     -406.367645
13     -955.972764
14    -1788.306085
15    -2780.076457
16    -3708.970456
17    -4349.718144
18    -4601.821227
20     -397.198381
21    -1239.624238
22    -2879.353002
23    -5360.900394
24    -8319.910103
25   -11090.569834
26   -13002.241271
27     -189.783610
30    -6344.832842
31   -11797.049889
32   -18300.523713
33   -24388.609072
34     -337.970840
35    -1541.239123
36    -4733.241010
40     -494.953978
41    -2234.515564
42    -6848.445458
43   -15826.622978
44   -29411.047880
45     -629.229451
50    -3212.152068
51    -9834.918236
52     -749.539471
53    -3355.163896
54     -748.375288
dtype: float64

In [24]:
V

weights    0.0
0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8          0.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14         0.0
15         0.0
16         0.0
17         0.0
18         0.0
20         0.0
21         0.0
22         0.0
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
30         0.0
31         0.0
32         0.0
33         0.0
34         0.0
35         0.0
36         0.0
40         0.0
41         0.0
42         0.0
43         0.0
44         0.0
45         0.0
50         0.0
51         0.0
52         0.0
53         0.0
54         0.0
dtype: float64

In [25]:
W

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [26]:
r.icrfs._StandardError()

ValueError: operands could not be broadcast together with shapes (55,55) (46,) 

In [ ]:
r.icrfs._CalcHeteroAdj()

LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [ ]:
para = r.icrfs.GetParameters().groupby('param_type')['param'].cumsum()
para

# model.feature_names_in_

0     1.950833
1     1.486848
2     2.603014
3     3.439503
4     4.058836
5     4.497487
6     4.784502
7     4.943370
8     4.999521
9     4.999521
10    1.486848
11    2.579745
12    3.367121
13    3.911446
14    4.279903
15    4.514840
16    4.641136
17    4.684375
18    4.684375
Name: param, dtype: float64

In [ ]:
r.icrfs._ProcessVarUBE().hist()

In [ ]:
r.icrfs.model

ElasticNet(alpha=3, fit_intercept=False, l1_ratio=0.05, max_iter=100000)

In [ ]:
r.icrfs.model.coef_

array([1.95083305, 1.48684759, 1.11616646, 0.83648878, 0.6193328 ,
       0.43865107, 0.28701556, 0.15886801, 0.056151  , 0.        ,
       1.48684788, 1.09289731, 0.78737558, 0.54432509, 0.36845754,
       0.23493658, 0.12629572, 0.04323888, 0.        ])

In [ ]:
r.icrfs.PredictTriangle().pivot_table(index='accident_period', columns='development_period', values='yhat').round(0)

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001,7.0,31.0,93.0,204.0,352.0,508.0,643.0,729.0,761.0,761.0
2002,31.0,138.0,411.0,902.0,1555.0,2248.0,2843.0,3225.0,3368.0,3368.0
2003,95.0,420.0,1253.0,2754.0,4747.0,6862.0,8679.0,9848.0,10283.0,10283.0
2004,219.0,970.0,2893.0,6358.0,10958.0,15839.0,20034.0,22731.0,23735.0,23735.0
2005,407.0,1802.0,5374.0,11811.0,20356.0,29424.0,37217.0,42227.0,44092.0,44092.0
2006,632.0,2794.0,8334.0,18314.0,31564.0,45626.0,57708.0,65477.0,68370.0,68370.0
2007,842.0,3723.0,11104.0,24403.0,42057.0,60794.0,76893.0,87245.0,91100.0,91100.0
2008,987.0,4364.0,13016.0,28605.0,49299.0,71261.0,90133.0,102267.0,106786.0,106786.0
2009,1044.0,4616.0,13768.0,30257.0,52146.0,75377.0,95339.0,108173.0,112953.0,112953.0


In [ ]:
r.paid_loss.incr_triangle.astype(float).round(0)

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001,357848.0,766940.0,610542.0,482940.0,527326.0,574398.0,146342.0,139950.0,227229.0,67948.0
2002,352118.0,884021.0,933894.0,1183289.0,445745.0,320996.0,527804.0,266172.0,425046.0,NaN
2003,290507.0,1001799.0,926219.0,1016654.0,750816.0,146923.0,495992.0,280405.0,NaN,NaN
2004,310608.0,1108250.0,776189.0,1562400.0,272482.0,352053.0,206286.0,NaN,NaN,NaN
2005,443160.0,693190.0,991983.0,769488.0,504851.0,470639.0,NaN,NaN,NaN,NaN
2006,396132.0,937085.0,847498.0,805037.0,705960.0,NaN,NaN,NaN,NaN,NaN
2007,440832.0,847631.0,1131398.0,1063269.0,NaN,NaN,NaN,NaN,NaN,NaN
2008,359480.0,1061648.0,1443370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,376686.0,986608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r.paid_loss.incr_triangle

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001,357848.0,766940.0,610542.0,482940.0,527326.0,574398.0,146342.0,139950.0,227229.0,67948.0
2002,352118.0,884021.0,933894.0,1183289.0,445745.0,320996.0,527804.0,266172.0,425046.0,NaN
2003,290507.0,1001799.0,926219.0,1016654.0,750816.0,146923.0,495992.0,280405.0,NaN,NaN
2004,310608.0,1108250.0,776189.0,1562400.0,272482.0,352053.0,206286.0,NaN,NaN,NaN
2005,443160.0,693190.0,991983.0,769488.0,504851.0,470639.0,NaN,NaN,NaN,NaN
2006,396132.0,937085.0,847498.0,805037.0,705960.0,NaN,NaN,NaN,NaN,NaN
2007,440832.0,847631.0,1131398.0,1063269.0,NaN,NaN,NaN,NaN,NaN,NaN
2008,359480.0,1061648.0,1443370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,376686.0,986608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r.icrfs.hetero_gp

,development_period,hetero_001,hetero_002,hetero_003,hetero_004,hetero_005,hetero_006,hetero_007,hetero_008,hetero_009,hetero_010
0,1,True,False,False,False,False,False,False,False,False,False
1,1,True,False,False,False,False,False,False,False,False,False
2,1,True,False,False,False,False,False,False,False,False,False
3,1,True,False,False,False,False,False,False,False,False,False
4,1,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
95,10,False,False,False,False,False,False,False,False,False,True
96,10,False,False,False,False,False,False,False,False,False,True
97,10,False,False,False,False,False,False,False,False,False,True
98,10,False,False,False,False,False,False,False,False,False,True


In [ ]:
r.icrfs.GetHeteroGp()

# r.icrfs.(r.icrfs.GetX().loc[r.icrfs.forecast_index])

,development_period
0,1
1,1
2,1
3,1
4,1
...,...
95,10
96,10
97,10
98,10


In [ ]:
r.paid_loss.ult('vwa', 5, tail=1.061)

Accident Period
2001-01-01    4139452.0
2002-01-01    5765176.0
2003-01-01    5706935.0
2004-01-01    5621078.0
2005-01-01    5154550.0
2006-01-01    5422953.0
2007-01-01    5961413.0
2008-01-01    7197996.0
2009-01-01    6120629.0
2010-01-01    5011529.0
Name: Chain Ladder Ultimate Loss, dtype: float64

In [ ]:
r.icrfs.Ultimate()

Accident Period
2001-01-01    282773.0
2002-01-01    211386.0
2003-01-01    244726.0
2004-01-01    176377.0
2005-01-01    215390.0
2006-01-01    258108.0
2007-01-01    256830.0
2008-01-01    325206.0
2009-01-01    389263.0
2010-01-01    418850.0
Name: Log-linear Ultimate, dtype: float64

In [ ]:
r.glm.Ultimate()

Accident Period
2001-01-01    6922300.0
2002-01-01    5969505.0
2003-01-01    5691169.0
2004-01-01    6683660.0
2005-01-01    5488934.0
2006-01-01    6131080.0
2007-01-01    7080337.0
2008-01-01    5750500.0
2009-01-01    6336181.0
2010-01-01    5896610.0
Name: tweedieGLM Ultimate, dtype: float64

In [ ]:
# chain ladder ult vs GLM
df = pd.concat([pd.DataFrame(r.paid_loss.ult()), pd.DataFrame(r.icrfs.Ultimate()), pd.DataFrame(r.glm.Ultimate())], axis=1)
df = df.assign(Diff=df['Log-linear Ultimate'] - df['Chain Ladder Ultimate Loss']).astype(int)
df['Diagonal'] = r.paid_loss.diag().astype(int)
df['Unpaid'] = df['Log-linear Ultimate'] - df.Diagonal
df

,Chain Ladder Ultimate Loss,Log-linear Ultimate,tweedieGLM Ultimate,Diff,Diagonal,Unpaid
Accident Period,,,,,,
2001-01-01,3901463,282773,6922300,-3618690,3901463,-3618690
2002-01-01,5433719,211386,5969505,-5222333,5339085,-5127699
2003-01-01,5378826,244726,5691169,-5134100,4909315,-4664589
2004-01-01,5297906,176377,6683660,-5121529,4588268,-4411891
2005-01-01,4858200,215390,5488934,-4642810,3873311,-3657921
2006-01-01,5111171,258108,6131080,-4853063,3691712,-3433604
2007-01-01,5660771,256830,7080337,-5403941,3483130,-3226300
2008-01-01,6784799,325206,5750500,-6459593,2864498,-2539292
2009-01-01,5642266,389263,6336181,-5253003,1363294,-974031


In [ ]:
r.icrfs.plot.obs_pred(log=True, color='dev')

In [ ]:
r.tweedie.plot.residual('acc', log=True)

AttributeError: 'rocky' object has no attribute 'tweedie'

In [ ]:
r.tweedie.plot.residual('dev', log=False)

: 

In [ ]:
r.paid_loss.tri

: 

In [ ]:
## accident period parameters should roughly follow the first column:
r.paid_loss.tri.iloc[:, 0].plot()

: 

In [ ]:
# but the actual fitted parameters include regularization terms that smooth out some of these bumps
r.tweedie.PlotParameter('acc')

: 

In [ ]:
# does not include dev_period_0012 - first development period is assumed to be included in the first accident period parameter
r.tweedie.PlotParameter('dev')

: 

In [ ]:
r.tweedie.Ultimate()

: 

In [ ]:
# final 2 accident years don't change much from 2020 - can set parameters for these years to be 0 (for no change)
r.tweedie.ManualFit(
  accident_period_2021=0, accident_period_2022=0,
  accident_period_2005=0,
  accident_period_2013=0,
  accident_period_2016=0,
  accident_period_2018=0
  )

: 

In [ ]:
r.tweedie.PlotParameter('acc')

: 

In [ ]:
r.tweedie.Ultimate()

: 

In [ ]:
## TODO
# - add a method to do a future calendar year projection
# - add a method to combine parameters into a single parameter for each year

: 

In [ ]:
# add a second glm model that includes calendar year effects
r.AddModel('tweediecal', 'glm', 'paid_loss', cal=True)
print(r)

: 

In [ ]:
# can look at design matrix for this model -- it has 39 more columns for the 39 calendar periods in a 20x20 triangle
r.tweediecal.GetX()

: 

In [ ]:
# using same hyperparameters as before, but could decide to refit
r.tweediecal.SetHyperparameters(alpha=0.3, power=1.8)
r.tweediecal

: 

In [ ]:
r.tweediecal.Fit()

: 

In [ ]:
r.tweediecal.Ultimate()

: 

In [ ]:
r.tweediecal.PlotParameter('acc')

: 

In [ ]:
r.tweediecal.PlotParameter('dev')

: 

In [ ]:
r.tweediecal.PlotParameter('cal')

: 

In [ ]:
## add a 1% future cal trend (eg assume that trend reverses itself)
future_cy_trend = 0.01
manfit = {}
for i in range(21, 40):
  manfit[f"calendar_period_00{i}"] = future_cy_trend

r.tweediecal.ManualFit(**manfit)

: 

In [ ]:
r.tweediecal.PlotParameter('cal')

: 

In [ ]:
r.tweediecal.Ultimate()

: 

In [ ]:
def parse_param_name(x):
  if x.find("_") == -1:
    return x
  else:
    end = x.split("_")[len(x.split("_"))-1]
    out = x.replace(f"_{end}", "")[:3]
    return out
  

df = pd.DataFrame(dict(nm=r.tweediecal.GetParameterNames(), val=r.tweediecal.model.coef_))
parse_param_name(df.nm[2])

: 

In [ ]:
df = pd.DataFrame({
  'chain ladder': r.paid_loss.ult(tail=1.05),
  'tweedie': r.tweedie.Ultimate(tail=1.05),
  'tweediecal': r.tweediecal.Ultimate(tail=1.05)
}).round(-3)/1000

df

: 

In [ ]:
r.Add

: 